In [59]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import bokeh.plotting
import bokeh.io
sys.path.insert(0, '../../')
import mwc.viz
import mwc.process
import mwc.stats
mwc.viz.pub_style()
bokeh.io.output_notebook()
%matplotlib inline

Loading BokehJS ...

In [63]:
# Load example data
fluct_data = pd.read_csv('../processing/microscopy/20181017_r1_37C_glycerol_O2_dilution/output/20181018_r1_37C_glycerol_O2_fluctuations.csv')
samples = pd.read_csv('../processing/microscopy/20181017_r1_37C_glycerol_O2_dilution/output/20181018_r1_37C_glycerol_O2_cal_factor_samples.csv')
fc_data = pd.read_csv('../processing/microscopy/20181017_r1_37C_glycerol_O2_dilution/output/20181018_r1_37C_glycerol_O2_foldchange.csv')

In [73]:
def dilution_summary(fluct_df, alpha_samples, fname=None, ip_dist=0.065):
    """Generates a Bokeh plot summarzing the dilution data."""
    # Compute necessary properties.
    fluct_df['I_1_sub'] = fluct_df['I_1'] - fluct_df['bg_val']
    fluct_df['I_2_sub'] = fluct_df['I_2'] - fluct_df['bg_val']
    
    # Determine if the fluctuations have been calculated
    if 'summed' not in fluct_df.keys():
        fluct_df['summed'] = fluct_df['I_1_sub'] * fluct_df['area_1'] +\
                        fluct_df['I_2_sub'] * fluct_df['area_2']
    if 'fluct' not in fluct_df.keys():
        fluct_df['fluct'] = (fluct_df['I_1_sub'] * fluct_df['area_1'] -\
                       fluct_df['I_2_sub'] * fluct_df['area_2'])**2
    
    # Compute the eCDF of fractional intensity. 
    frac_int = (fluct_df['I_1_sub'] * fluct_df['area_1']) / fluct_df['summed']
    x, ecdf = np.sort(frac_int), np.arange(0, len(frac_int), 1) / len(frac_int)
    
    # Bin the data by events. 
    binned = mwc.stats.bin_by_events(fluct_df, 50)
    
    # Generate the histogram of alpha samples
    hist, bins = np.histogram(alpha_samples, bins=100, density=True)
 
    # Compute the fit line
    hpd = mwc.stats.compute_hpd(alpha_samples, 0.95)
    summed_range = np.logspace(np.log10(np.min(fluct_df['summed'])) - 1,
                              np.log10(np.max(fluct_df['summed'])) + 1, 200)
    fit_lower = hpd[0] * summed_range
    fit_upper = hpd[1] * summed_range
    
    # Instantiate the figure canvas
    p1 = bokeh.plotting.figure(width=700, height=300,
                               x_axis_type='log', y_axis_type='log',
                               x_axis_label='summed intensity [a.u.]',
                               y_axis_label='fluctuations [a.u.^2]')
    p2 = bokeh.plotting.figure(width=300, height=200,
                               x_axis_label='cell area [µm^2]', 
                               y_axis_label='mean intensity [a.u. / pix]') 
    p3 = bokeh.plotting.figure(width=200, height=200, 
                               x_axis_label='partitioned intensity', 
                               y_axis_label='ECDF')
    p4 = bokeh.plotting.figure(width=200, height=200,
                              x_axis_label='calibration factor [a.u. / molecule]',
                              y_axis_label='~ probability')
    
    # Plot the fluctuation glyphs
    p1.circle(fluct_df['summed'], fluct_df['fluct'], size=1, color='black', legend='divisions')
    p1.circle(binned['summed'], binned['fluct'], size=4, color='tomato', legend='binned data (50 events)')
    p1.line(summed_range, fit_upper, color='dodgerblue')
    p1.line(summed_range, fit_lower, color='dodgerblue')
   
    # Plot the intensity versus area
    p2.circle(fluct_df['area_1'] * ip_dist**2, fluct_df['I_1'], size=1.5, color='black', legend='I_1')
    p2.circle(fluct_df['area_2'] * ip_dist**2, fluct_df['I_2'], size=1.5, color='tomato', legend='I_2')
    
    # Plot the ecdf of partitioned intensity
    p3.line(x, ecdf, color='black', line_width=2)
    
    # Plot the cal factor distribution. 
    p4.step(bins[:-1], hist, color='black')
    
    # Create the layout and return
    _layout = bokeh.layouts.gridplot([[p1], [p2, p3, p4]])
     
    # Save the figure as html and png. 
    bokeh.io.export_png(_layout, f"{fname.split('.')[0]}.png")
    bokeh.io.save(_layout, f"{fname.split('.')[0]}.html")
    return _layout
                        
_ = dilution_summary(fluct_data, samples['alpha'], 'a')

bokeh.io.show(_)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in log10
/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:136: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


Make a fold-change meausurement summary plot 

In [153]:
# Compute the repressor values from samples. 
def fold_change_summary_microscopy(fc_data, alpha_samples, fname=None, constants=None, operator='O2'):
    """Generates a summary plot of measured fold-change"""    
    
    # Compute the credible regions and median for repressors per cell
    hpd = compute_hpd(alpha_samples['alpha'], 0.95)
    int_mCherry = (fc_data['mean_mCherry'] - fc_data['mCherry_bg_val']) * fc_data['area_pix'] 
    fc_data['repressors_min'] = int_mCherry / hpd[0]
    fc_data['repressors_max'] = int_mCherry / hpd[1]
    fc_data['repressors_median'] = int_mCherry / np.median(alpha_samples['alpha'])

    # Compute the theory line
    rep_range = np.logspace(0, 4, 200)
    arch = SimpleRepression(rep_range, ep_r=constants[operator], ep_ai=constants['ep_ai'],
                                 n_ns=constants['n_ns'], ka=constants['ka'], ki=constants['ki'],
                                 n_sites=constants['n_sites'], effector_conc=0).fold_change()

    # Define the color palettes
    mch_colors = {conc: color for conc, color in zip(np.sort(fc_data['atc_ngml'].unique()), bokeh.palettes.Reds9)}
    yfp_colors = {conc: color for conc, color in zip(np.sort(fc_data['atc_ngml'].unique()), bokeh.palettes.Greens9)}

    # Instantiate the figure canvases
    p1 = bokeh.plotting.figure(width=350, height=250, 
                      x_axis_label='mean mCherry intensity [a.u. / pix]',
                      y_axis_label='ECDF', x_axis_type='log', y_axis_type='log')
    p2 = bokeh.plotting.figure(width=350, height=250, 
                      x_axis_label='mean YFP intensity [a.u. / pix]',
                      y_axis_label='ECDF', x_axis_type='log')
    p3 = bokeh.plotting.figure(width=350, height=250, x_axis_label='ATC [ng / mL]',
                          y_axis_label='rep. per cell')
    p4 = bokeh.plotting.figure(width=350, height=250, x_axis_type='log', y_axis_type='log',
                           x_axis_label='rep. per cell', y_axis_label='fold-change')


    for g, d in fc_data[fc_data['strain']=='dilution'].groupby(['atc_ngml']):
        # Compute the ECDF of mCherry intensity
        x_mch, y_mch = np.sort(d['mean_mCherry']), np.arange(0, len(d), 1) /len(d)    
    
        # Plot the various glyphs
        p1.circle(d['mean_mCherry'], d['mean_yfp'], color=mch_colors[g], size=0.75, legend=str(g))
        p2.line(x_mch, y_mch, color=mch_colors[g], line_width=2, legend=str(g))
        p3.circle(g, np.mean(d['repressors_median']))
        p3.line([g,g],  [np.mean(d['repressors_min']), np.mean(d['repressors_max'])])
        p4.circle(np.mean(d['repressors_median']), np.mean(d['fold_change']))
        p4.line(rep_range, arch, line_width=2, color='black')
    
    # Assemble the layout and determine if the plot should be saved
    _layout = bokeh.layouts.gridplot([[p1, p2], [p3, p4]])
    if fname != None:
        bokeh.io.export_png(_layout, f"output/{fname.split('.')[0]}.png")
        bokeh.io.save(_layout, f"output/{fname.split('.')}.html", title=fname)
    return _layout

bokeh.io.show(_layout)

In [152]:
np.mean(d['fold_change'])

0.01283006831138436

{0.0: '#67000d',
 1.0: '#a50f15',
 2.0: '#cb181d',
 3.0: '#ef3b2c',
 4.0: '#fb6a4a',
 7.0: '#fc9272',
 10.0: '#fcbba1'}

In [91]:
a = fc_data.groupby(['strain', 'atc_ngml']).apply(_compute_ecdf)

In [99]:
a.reset_index()

,strain,atc_ngml,x_mcherry,x_yfp,ecdf_mcherry,ecdf_yfp
0,auto,0.0,"[166.93718592964825, 167.77221324717286, 169.9...","[139.00502512562815, 140.91760904684975, 141.4...","[0.0, 0.0007331378299120235, 0.001466275659824...","[0.0, 0.0007331378299120235, 0.001466275659824..."
1,delta,0.0,"[173.32781456953643, 175.61714285714282, 175.8...","[149.89670658682638, 153.6997245179063, 154.32...","[0.0, 0.0017301038062283738, 0.003460207612456...","[0.0, 0.0017301038062283738, 0.003460207612456..."
2,dilution,1.0,"[174.0492505353319, 181.2647887323944, 181.771...","[143.76481835564053, 145.0480769230769, 145.71...","[0.0, 0.0015625, 0.003125, 0.0046875, 0.00625,...","[0.0, 0.0015625, 0.003125, 0.0046875, 0.00625,..."
3,dilution,2.0,"[175.51146384479716, 186.80640465793306, 191.5...","[141.25908372827803, 141.26589595375722, 141.4...","[0.0, 0.0018281535648994515, 0.003656307129798...","[0.0, 0.0018281535648994515, 0.003656307129798..."
4,dilution,3.0,"[199.49897330595482, 204.5019305019305, 204.99...","[141.0664652567976, 142.06884681583475, 142.30...","[0.0, 0.001953125, 0.00390625, 0.005859375, 0....","[0.0, 0.001953125, 0.00390625, 0.005859375, 0...."
5,dilution,4.0,"[186.06040268456374, 188.98445595854923, 189.3...","[140.4988814317673, 140.97487437185933, 141.07...","[0.0, 0.0014184397163120568, 0.002836879432624...","[0.0, 0.0014184397163120568, 0.002836879432624..."
6,dilution,7.0,"[173.49184782608697, 181.20758483033933, 182.1...","[140.74184782608697, 140.85822784810128, 141.2...","[0.0, 0.0017452006980802793, 0.003490401396160...","[0.0, 0.0017452006980802793, 0.003490401396160..."
7,dilution,10.0,"[174.23471400394473, 180.12305295950156, 180.8...","[139.44773175542406, 139.6651090342679, 141.02...","[0.0, 0.0012254901960784314, 0.002450980392156...","[0.0, 0.0012254901960784314, 0.002450980392156..."
